En este experimento se busca simular una base de conversaciones para luego probar la capacidad del asistente de navegar cronológicamente la base y comenzar a usarlas en un razonamiento.

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [27]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()

db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


Se crea una base de mensajes entre el asistente y el usuario.

In [106]:
conversation_db =  [
    {
        "message": {
            "role": "user",
            "content": "Cual es la capital de Argentina?"
        },
        "metadata": {
          "session_id": "1",
          "person_id": "20",
          "message_id": "1-1",
          "created_at": "20230825T12:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "Buenos Aires"
        },
        "metadata": {
            "session_id": "1",
            "person_id": "20",
            "message_id": "1-2",
            "created_at": "20230825T12:01:00.000Z"
        }
    },
    {
        "message": {
            "role": "user",
            "content": "Cual es la capital de Chile?"
        },
        "metadata": {
          "session_id": "2",
          "person_id": "20",
          "message_id": "2-1",
          "created_at": "20230825T13:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "Santiago"
        },
        "metadata": {
            "session_id": "2",
            "person_id": "20",
            "message_id": "2-2",
            "created_at": "20230825T13:01:00.000Z"
        }
    },
    {
        "message": {
            "role": "user",
            "content": "Cual es la capital de Peru?"
        },
        "metadata": {
          "session_id": "3",
          "person_id": "20",
          "message_id": "3-1",
          "created_at": "20230825T14:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "Lima"
        },
        "metadata": {
            "session_id": "3",
            "person_id": "20",
            "message_id": "3-2",
            "created_at": "20230825T14:01:00.000Z"
        }
    },
    {
        "message": {
            "role": "user",
            "content": "Cual es la capital de Colombia?"
        },
        "metadata": {
          "session_id": "4",
          "person_id": "20",
          "message_id": "4-1",
          "created_at": "20230825T15:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "Bogota"
        },
        "metadata": {
            "session_id": "4",
            "person_id": "20",
            "message_id": "4-2",
            "created_at": "20230825T15:01:00.000Z"
        }
    }

  ]



In [32]:
conversation_db[0]['message']

{'role': 'user', 'content': 'Cual es la capital de Argentina?'}

In [17]:
str(conversation_db[0]['message'])


"{'role': 'user', 'content': 'Cual es la capital de Argentina?'}"

Los mensajes se vectorizan y se suben a la base

In [33]:
for item in conversation_db:
    print(str(item['message']))
    print(item['metadata'])
    db.add_texts(
        texts=[str(item['message'])], 
        metadatas=[item['metadata']],
    )

{'role': 'user', 'content': 'Cual es la capital de Argentina?'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-1', 'created_at': '20230825T12:00:00.000Z'}
{'role': 'system', 'content': 'Buenos Aires'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-2', 'created_at': '20230825T12:01:00.000Z'}
{'role': 'user', 'content': 'Cual es la capital de Chile?'}
{'session_id': '2', 'person_id': '20', 'message_id': '2-1', 'created_at': '20230825T13:00:00.000Z'}
{'role': 'system', 'content': 'Santiago'}
{'session_id': '2', 'person_id': '20', 'message_id': '2-2', 'created_at': '20230825T13:01:00.000Z'}
{'role': 'user', 'content': 'Cual es la capital de Peru?'}
{'session_id': '3', 'person_id': '20', 'message_id': '3-1', 'created_at': '20230825T14:00:00.000Z'}
{'role': 'system', 'content': 'Lima'}
{'session_id': '3', 'person_id': '20', 'message_id': '3-2', 'created_at': '20230825T14:01:00.000Z'}
{'role': 'user', 'content': 'Cual es la capital de Colombia?'}
{'session_id': '4', 'person_

In [50]:
db_results[0][0].page_content


"{'role': 'user', 'content': 'Cual es la capital de Peru?'}"

In [52]:
str(db_results[0][0].metadata)


"{'session_id': '3', 'person_id': '20', 'message_id': '3-1', 'created_at': '20230825T14:00:00.000Z'}"

In [56]:
context = "{ message: " + db_results[0][0].page_content + ", metadata: " + str(db_results[0][0].metadata) + " }"
context

"{ message: {'role': 'user', 'content': 'Cual es la capital de Peru?'}, metadata: {'session_id': '3', 'person_id': '20', 'message_id': '3-1', 'created_at': '20230825T14:00:00.000Z'} }"

In [64]:
prompt = f"""
Answer the user's question this message as context.
Previous message: {context}
"""



In [65]:
prompt

"\nAnswer the user's question this message as context.\nContext: { message: {'role': 'user', 'content': 'Cual es la capital de Peru?'}, metadata: {'session_id': '3', 'person_id': '20', 'message_id': '3-1', 'created_at': '20230825T14:00:00.000Z'} }\n"

Se crea la función para llamar al LLM

In [96]:
# Install the Python Requests library:
# `pip install requests`

import requests
import json


def gpt35_system_user(system_message:str, user_message:str):
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions",
            params={
                "api-version": "2023-05-15",
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "api-key": "bd6603c266fc49389659225a451ff03d",
            },
            data=json.dumps({
                "messages": [
                    {
                        "content": system_message,
                        "role": "system"
                    },
                    {
                        "content":user_message,
                        "role": "user"
                    }
                ]
            })
        )
        choices = response.json()['choices']
        answer = {
            "status_code": response.status_code,
            "text": choices[0]['message'],
        }
        return answer

    except requests.exceptions.RequestException:
        print('HTTP Request failed')
        return None


Se supone un nuevo mensaje del usuairo

In [68]:
user_input_1 = "hace cuanto hablamos de peru?"

In [88]:
system_prompt_1 = f"""
Answer the user's question. Use this message as context.
Previous message: {context_1}
"""
system_prompt_1

"\nAnswer the user's question this message as context.\nPrevious message: { message: {'role': 'user', 'content': 'Cual es la capital de Peru?'}, metadata: {'session_id': '3', 'person_id': '20', 'message_id': '3-1', 'created_at': '20230825T14:00:00.000Z'} }\n"

El modelo de lenguaje toma el contexto proporcionado por la base de conversaciones antes de generar la respuesta.

In [97]:
answer_1 = gpt35_system_user(system_message=system_prompt_1, user_message=user_input)
answer_1["text"]["content"]

'Según la información proporcionada en el mensaje anterior, empezamos a hablar de Perú en el mensaje con identificador "3-1" que fue creado el día 25 de agosto del año 2308 a las 14:00:00 (en formato UTC).'

In [103]:
user_input_1 = "te pregunte por brasil?"
filter = {"person_id": "20"}
db_results_1 = db.similarity_search_with_score(query=user_input_1, embeddings=embeddings, filter=filter, k=1)
context_1 = "{ message: " + db_results_1[0][0].page_content + ", metadata: " + str(db_results_1[0][0].metadata) + " }"
system_prompt_1 = f"""
Answer the user's question this message as context.
Previous message: {context_1}
"""
answer_1 = gpt35_system_user(system_message=system_prompt_1, user_message=user_input_1)
answer_1["text"]["content"]

'Lo siento, pero no veo ninguna pregunta en el mensaje anterior. ¿Puedo ayudarte en algo más?'

In [104]:
user_input_2 = "alguna vez hablamos de colombia no? cuando fue eso?"
filter = {"person_id": "20"}
db_results_2 = db.similarity_search_with_score(query=user_input_2, embeddings=embeddings, filter=filter, k=1)
context_2 = "{ message: " + db_results_2[0][0].page_content + ", metadata: " + str(db_results_2[0][0].metadata) + " }"
print(f"CONTEXT:{context_2}")
system_prompt_2 = f"""
You are a personal assistant. 
You have access to all previous conversations with your user.
All your responses should be based on the previous message provided.
Previous message: {context_2}
"""
answer_2 = gpt35_system_user(system_message=system_prompt_2, user_message=user_input_2)
answer_2["text"]["content"]


CONTEXT:{ message: {'role': 'user', 'content': 'Cual es la capital de Colombia?'}, metadata: {'session_id': '4', 'person_id': '20', 'message_id': '4-1', 'created_at': '20230825T15:00:00.000Z'} }


'Sí, hemos hablado sobre Colombia antes. La última vez que hablamos sobre Colombia fue en la sesión 4, cuando me preguntaste cuál era la capital de ese país. ¿Hay algo más en lo que pueda ayudarte?'

In [105]:
user_input_3 = "hace cuanto te habia preguntado por argentina?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant.
You have access to all previous conversations with your user.
All your responses should be based on the previous message provided.
Previous message: {context_3}
"""
answer_3 = gpt35_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: {'role': 'user', 'content': 'Cual es la capital de Argentina?'}, metadata: {'session_id': '1', 'person_id': '20', 'message_id': '1-1', 'created_at': '20230825T12:00:00.000Z'} }


'Según la metadata de la conversación, me preguntaste sobre la capital de Argentina el 25 de Agosto del 2023 a las 12:00:00 PM (UTC). ¿Hay algo más en lo que te pueda ayudar?'